<a href="https://colab.research.google.com/github/XShen72/deeplearning.ai-Assignments/blob/master/spark_basic_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!file /content/spark-3.5.0-bin-hadoop3.tgz
!tar zxvf /content/spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

/content/spark-3.5.0-bin-hadoop3.tgz: gzip compressed data, from Unix, original size modulo 2^32 445460480
spark-3.5.0-bin-hadoop3/
spark-3.5.0-bin-hadoop3/kubernetes/
spark-3.5.0-bin-hadoop3/kubernetes/tests/
spark-3.5.0-bin-hadoop3/kubernetes/tests/pyfiles.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/autoscale.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/python_executable_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/worker_memory_check.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/py_container_checks.py
spark-3.5.0-bin-hadoop3/kubernetes/tests/decommissioning_cleanup.py
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/decom.sh
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/
spark-3.5.0-bin-hadoop3/kubernetes/dockerfiles/spark/bindings/R/Dock

In [29]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [31]:
import findspark
findspark.init()
from pyspark import SparkContext
sc = SparkContext()

# Resilient Distributed Datasets (RDDs)
Spark revolves around the concept of a resilient distributed dataset (RDD), which is a fault-tolerant collection of elements that can be operated on in parallel. There are two ways to create RDDs: parallelizing an existing collection in your driver program, or referencing a dataset in an external storage system, such as a shared filesystem, HDFS, HBase, or any data source offering a Hadoop InputFormat.

In [36]:
data = [1, 2, 3, 4, 5]
distData = sc.parallelize(data)

s = distData.reduce(lambda a, b : a*b)

print (s)

#.textfile()


120


PythonRDD[10] at RDD at PythonRDD.scala:53


Once created, the distributed dataset (distData) can be operated on in parallel. For example, we can call distData.reduce(lambda a, b: a + b) to add up the elements of the list. We describe operations on distributed datasets later on.

One important parameter for parallel collections is the number of partitions to cut the dataset into. Spark will run one task for each partition of the cluster. Typically you want 2-4 partitions for each CPU in your cluster. Normally, Spark tries to set the number of partitions automatically based on your cluster. However, you can also set it manually by passing it as a second parameter to parallelize (e.g. sc.parallelize(data, 10)). Note: some places in the code use the term slices (a synonym for partitions) to maintain backward compatibility.

# RDD Operations

RDDs support two types of operations: transformations, which create a new dataset from an existing one, and actions, which return a value to the driver program after running a computation on the dataset. For example, map is a transformation that passes each dataset element through a function and returns a new RDD representing the results. On the other hand, reduce is an action that aggregates all the elements of the RDD using some function and returns the final result to the driver program (although there is also a parallel reduceByKey that returns a distributed dataset).

All transformations in Spark are lazy, in that they do not compute their results right away. Instead, they just remember the transformations applied to some base dataset (e.g. a file). The transformations are only computed when an action requires a result to be returned to the driver program. This design enables Spark to run more efficiently. For example, we can realize that a dataset created through map will be used in a reduce and return only the result of the reduce to the driver, rather than the larger mapped dataset.

By default, each transformed RDD may be recomputed each time you run an action on it. However, you may also persist an RDD in memory using the persist (or cache) method, in which case Spark will keep the elements around on the cluster for much faster access the next time you query it. There is also support for persisting RDDs on disk, or replicated across multiple nodes.

# Basics

To illustrate RDD basics, consider the simple program below:

In [45]:
def transformFunc(line):
  return len(line)

def reduceFunc(a, b):
  return a+b

# Transform: map, filter, FlatMap
# Action     reduce

#lines = sc.textFile("exa.txt")



file = ["line1", "This is line2"]
lines = sc.parallelize(file)
print (lines.collect())
lineLengths = lines.map(lambda line : len(line))

print (lineLengths.collect())
totalLength = lineLengths.reduce(reduceFunc)
print (totalLength)

# transform     map filter flatmap
# action        reduce, collect

['line1', 'This is line2']
[5, 13]
18


The first line defines a base RDD from an external file. This dataset is not loaded in memory or otherwise acted on: lines is merely a pointer to the file. The second line defines lineLengths as the result of a map transformation. Again, lineLengths is not immediately computed, due to laziness. Finally, we run reduce, which is an action. At this point Spark breaks the computation into tasks to run on separate machines, and each machine runs both its part of the map and a local reduction, returning only its answer to the driver program.

If we also wanted to use lineLengths again later, we could add:

In [46]:
lineLengths.persist()

PythonRDD[18] at collect at <ipython-input-45-44fa61abb3bc>:19

before the reduce, which would cause lineLengths to be saved in memory after the first time it is computed.

# Passing Functions to Spark
Spark’s API relies heavily on passing functions in the driver program to run on the cluster. There are three recommended ways to do this:

* Lambda expressions, for simple functions that can be written as an expression. (Lambdas do not support multi-statement functions or statements that do not return a value.)
* Local defs inside the function calling into Spark, for longer code.
* Top-level functions in a module.

For example, to pass a longer function than can be supported using a lambda, consider the code below:

In [48]:
"""MyScript.py"""
if __name__ == "__main__":
    def myFunc(s):
        words = s.split(" ")
        return len(words)

    sc = SparkContext(...)
    sc.textFile("file.txt").map(myFunc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at <ipython-input-31-f2ddc2e94d00>:4 

# Working with Key-Value Pairs

While most Spark operations work on RDDs containing any type of objects, a few special operations are only available on RDDs of key-value pairs. The most common ones are distributed “shuffle” operations, such as grouping or aggregating the elements by a key.

In Python, these operations work on RDDs containing built-in Python tuples such as (1, 2). Simply create such tuples and then call your desired operation.

For example, the following code uses the reduceByKey operation on key-value pairs to count how many times each line of text occurs in a file:

In [ ]:
file = ["line1", "This is line2"]
lines = sc.parallelize(file)
print (lines.collect())
lineLengths = lines.map(lambda line : len(line))

In [54]:
words = ["hello this is line one", "hello this is line two"]
words_rdd = sc.parallelize(words)
print (words_rdd.collect())
words_rdd = words_rdd.flatMap(lambda line: line.split(" "))
print (words_rdd.collect())
pairs = words_rdd.map(lambda s: (s, 1))
print (pairs.collect())
counts = pairs.reduceByKey(lambda a, b: a + b)
print (counts.collect())

['hello this is line one', 'hello this is line two']
['hello', 'this', 'is', 'line', 'one', 'hello', 'this', 'is', 'line', 'two']
[('hello', 1), ('this', 1), ('is', 1), ('line', 1), ('one', 1), ('hello', 1), ('this', 1), ('is', 1), ('line', 1), ('two', 1)]
[('this', 2), ('is', 2), ('line', 2), ('two', 1), ('hello', 2), ('one', 1)]


#RDD API Examples

##Word Count
In this example, we use a few transformations to build a dataset of (String, Int) pairs called counts and then save it to a file.

In [56]:
# text_file = sc.textFile("hdfs://...")
from google.colab import files
files.upload()

text_file = sc.textFile("file.txt")

Saving file.txt to file (1).txt


In [57]:
print (text_file.collect())

['Quod equidem non reprehendo;', 'Lorem ipsum dolor sit amet, consectetur adipiscing elit. Quibus natura iure responderit non esse verum aliunde finem beate vivendi, a se principia rei gerendae peti; Quae enim adhuc protulisti, popularia sunt, ego autem a te elegantiora desidero. Duo Reges: constructio interrete. Tum Lucius: Mihi vero ista valde probata sunt, quod item fratri puto. Bestiarum vero nullum iudicium puto. Nihil enim iam habes, quod ad corpus referas; Deinde prima illa, quae in congressu solemus: Quid tu, inquit, huc? Et homini, qui ceteris animantibus plurimum praestat, praecipue a natura nihil datum esse dicemus?', '', 'Iam id ipsum absurdum, maximum malum neglegi. Quod ea non occurrentia fingunt, vincunt Aristonem; Atqui perspicuum est hominem e corpore animoque constare, cum primae sint animi partes, secundae corporis. Fieri, inquam, Triari, nullo pacto potest, ut non dicas, quid non probes eius, a quo dissentias. Equidem e Cn. An dubium est, quin virtus ita maximam par

In [59]:
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

for x in counts.collect():
  print (x)

('Quod', 4)
('Lorem', 1)
('ipsum', 3)
('consectetur', 1)
('elit.', 1)
('natura', 3)
('responderit', 1)
('verum', 2)
('aliunde', 1)
('finem', 1)
('beate', 2)
('rei', 1)
('gerendae', 1)
('adhuc', 1)
('sunt,', 3)
('desidero.', 1)
('Duo', 1)
('interrete.', 1)
('Mihi', 1)
('vero', 3)
('valde', 1)
('probata', 1)
('item', 1)
('fratri', 1)
('puto.', 2)
('nullum', 1)
('iam', 1)
('Deinde', 1)
('quae', 3)
('in', 5)
('tu,', 1)
('inquit,', 1)
('huc?', 1)
('praestat,', 1)
('praecipue', 1)
('dicemus?', 1)
('', 5)
('malum', 1)
('neglegi.', 1)
('fingunt,', 1)
('vincunt', 1)
('est', 3)
('animoque', 1)
('constare,', 1)
('sint', 1)
('partes,', 1)
('secundae', 1)
('Fieri,', 1)
('pacto', 1)
('quid', 2)
('Equidem', 1)
('est,', 4)
('quin', 1)
('virtus', 2)
('ita', 5)
('maximam', 1)
('partem', 1)
('optineat', 1)
('Summus', 1)
('plures', 1)
('dies', 1)
('Dicet', 1)
('ipsa', 3)
('dubitabit', 1)
('beato', 1)
('fuisse,', 1)
('illum,', 1)
('is', 1)
('condemnatus', 1)
('rogatione,', 1)
('P.', 1)
('si', 4)
('cur', 1)

## Pi Estimation
Spark can also be used for compute-intensive tasks. This code estimates π by "throwing darts" at a circle. We pick random points in the unit square ((0, 0) to (1,1)) and see how many fall in the unit circle. The fraction should be π / 4, so we use this to get our estimate.

In [61]:
import random

def inside(p):
    x, y = random.random(), random.random()
    return x*x + y*y < 1

NUM_SAMPLES = 10000000
count = sc.parallelize(range(0, NUM_SAMPLES)) \
             .filter(inside).count()

print("Pi is roughly %f" % (4.0 * count / NUM_SAMPLES))

Pi is roughly 3.141291


#DataFrame API Examples
In Spark, a DataFrame is a distributed collection of data organized into named columns. Users can use DataFrame API to perform various relational operations on both external data sources and Spark’s built-in distributed collections without providing specific procedures for processing data. Also, programs based on DataFrame API will be automatically optimized by Spark’s built-in optimizer, Catalyst.

##Text Search
In this example, we search through the error messages in a log file.

In [69]:
from pyspark.sql import SparkSession

In [70]:
print (hasattr(text_file, "toDF"))
# False

SparkSession(sc)
print (hasattr(text_file, "toDF"))
## True

True
True


In [67]:
df = sc.parallelize([("Tom", "M", 20), ("Jack", "M", 18), ("Marry", "F", 20)])\
       .toDF(['name', 'sex', 'age'])

df.show()

+-----+---+---+
| name|sex|age|
+-----+---+---+
|  Tom|  M| 20|
| Jack|  M| 18|
|Marry|  F| 20|
+-----+---+---+



In [68]:
# Counts by certain column
countsByCol = df.groupBy("age").count()
countsByCol.show()

+---+-----+
|age|count|
+---+-----+
| 20|    2|
| 18|    1|
+---+-----+



In [71]:
from pyspark.sql.functions import col

errors = df.filter(col("age") > 19)
errors.show()

+-----+---+---+
| name|sex|age|
+-----+---+---+
|  Tom|  M| 20|
|Marry|  F| 20|
+-----+---+---+



In [72]:
# Creates a DataFrame having a single column named "line"
df = text_file.map(lambda k: k.split("\\t")).toDF(["line"])

errors = df.filter(col("line").like("%ERROR%"))
# Counts all the errors
print (errors.count())

0
